In [9]:
#%%writefile scripts/story_placements.py
#!/usr/bin/env python

import pandas as pd
from pathlib import Path
from storage.cherrytree import CherryTree
import sys
import redis
import dateparser

def story_placements(story_node, scenes_node, index_file):
    ct = CherryTree(index_file)
    story_node = ct.find_node_by_name(story_node)
    scene_node = ct.find_node_by_name(scenes_node)
    story_links = pd.DataFrame([dict(Scene=n.name, href=l.href, sc_no=no)
                                for no, n in enumerate(scene_node.descendants)
                                for l in n.links if l.type == 'node'])
    story_nodes = pd.DataFrame([dict(Story=n.name, href=n.id, st_no=no)
                                for no, n in enumerate(story_node.descendants)])

    df = story_nodes.merge(story_links, how='left', on='href')
    return df.fillna('Unplaced')[['Story', 'Scene']]


In [5]:
ct = CherryTree('synopsis.ctd')
r = redis.Redis(decode_responses=True)

In [29]:
df = pd.DataFrame([r.hgetall(k) for k in r.keys('omaf*')]).fillna('No Data')

In [30]:
df['timestamp'] = df.date.apply(lambda x: dateparser.parse(x))

In [ ]:

[s.name for s in ct.nodes('Scenes') \
     if not next((l for l in s.links if  l.type == 'node'), None) if s.filepath]
    

In [3]:
[n.name for n in ct.nodes() if n.filepath if 'stories' in n.filepath]

['VTCLA Crashes', 'Banteng Merah', 'Ambarrawa Raid']

In [ ]:
scene_node = 'Scenes'
index_file = 'synopsis.ctd'

In [ ]:
story_node = 'Enduring Friendship'
story_placements(story_node, scene_node, index_file)

In [ ]:
[s.name for s in ct.nodes() if s.filepath \
  if 'scenes' in s.filepath \
  if not next((a for a in s.ancestors if a.name == 'Scenes'), None)]
             

In [39]:
df.sort_values('seq')[['title', 'seq']]

,title,seq
111,Sabam Assigns Muharto Interview,0
44,Cameron Interviews Muharto,1
98,Beach Landing,10
24,SE Air Bankrupt,100
53,Spite Their Face,101
...,...,...
54,Bob Returns,95
64,POAS Flights,96
117,Spoor Orders Night Patrol,97
6,Partner Has Done Nothing,98


In [35]:
df.seq

0       91
1       72
2       12
3       21
4       90
      ... 
122     49
123     20
124    116
125     79
126     65
Name: seq, Length: 127, dtype: object